# Consideraciones para el entrenamiento

En el punto 5 del paper hablan de cómo han entrenado al transformer, así que vamos a verlo

## Datos

Para entrenar al transformer dicen que han usado el dataset [WMT 2014 English-German](https://aclanthology.org/W14-3302.pdf), pero como hemos instalado la librería de transformers de huggingface, vamos a usar su librería datasets para usar el dataset [opus100](https://huggingface.co/datasets/opus100) del inglés al español (`en-es`)

## Hardware

Han usado 8 Nvidia P100. Supongo que ninguno de nosotros tenemos acceso a 8 GPUs de ese tipo, por lo que haremos un código que sea ejecutable en [Colab](https://colab.research.google.com/), que al día de escribir este curso ofrece GPUs como la Nvidia T4 que tiene la misma VRAM que la P100, pero con una arquitectura más nueva. Por lo que en vez de usar 8 GPUs como las del paper, usaremos una un poco mejor que una de las que usaron ellos

## Steps

Dicen que entrenaron el modelo 100.000 pasos o 12 horas

## Optimizador

Los investigadores usaron un optimizador Adam con $\beta_1 = 0.9$, $\beta_2 = 0.98$ y $\epsilon = 10^{-9}$ y que variaron el learning rate según la fórmula

$$lr = d_{model}^{-0.5} \cdot min \left(stepNumber^{-0.5}, stepNumber \cdot stepWarmUp^{-1.5} \right)$$

Esto corresponde a un aumento lineal de la tasa de aprendizaje para los primeros pasos de entrenamiento (pasos de calentamiento (warmup)), y a una disminución posterior proporcional a la raíz cuadrada inversa del número de pasos. Utilizaron 4000 pasos de calentamiento

## Regularización

Durante el entrenamiento utilizaron dropout con una probabilidad de 0.1 y label smoothing